1.
F-test
AIC

2.
Third Derivate of g is continuous

3.
Cubic Spline
Natural Cubic Spline

4. 2
5. -1
6. -3
7. NA
8. 1.93
9. 0.44
10. Yes
11. 1.93
12. 0.44
13. No

14.
bs(+1, Intercept)
poly

15.
bs(+1, Intercept)
Median

16.
When lambda is Infinity, = Linear Regression
When lambda is Zero, It passes through all data points
Fitted curve is NCS

In [334]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots

import statsmodels.api as sm

from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF
from statsmodels.stats.anova import anova_lm

from ISLP import load_data
from ISLP.models import bs, ns

from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)

In [203]:
from pygam import (s as s_gam,
                   l as l_gam,
                   f as f_gam,
                   LinearGAM,
                   LogisticGAM)

from ISLP.transforms import (BSpline,
                             NaturalSpline)
from ISLP.models import bs, ns
from ISLP.pygam import (approx_lam,
                        degrees_of_freedom,
                        plot as plot_gam,
                        anova as anova_gam)

In [204]:
myData = pd.read_csv("https://liangfgithub.github.io/Data/noxData.csv")

In [205]:
myData

x = myData['dis']
y = myData['nox']

In [373]:
#model = 'act_hours ~ h_hours + I(h_hours**2)'

#sm.OLS(y, x).fit().summary()
#dir(poly)
#poly_x

print(np.median(x))
print(np.quantile(x, q=[.25, 0.5, .75]))

3.2074499999999997
[2.100175 3.20745  5.188425]


In [384]:
#bs_ = BSpline(internal_knots=[25,40,60], intercept=True).fit(age)
bs_ = BSpline(df=7, intercept=True).fit(x)

print(bs_.internal_knots_)
print(bs_.transform(x))
#x


[2.100175 3.20745  5.188425]
     BSpline(df=7, intercept=True, lower_bound=1.1296, upper_bound=12.1265)[0]  \
0                                                  0.0                           
1                                                  0.0                           
2                                                  0.0                           
3                                                  0.0                           
4                                                  0.0                           
..                                                 ...                           
501                                                0.0                           
502                                                0.0                           
503                                                0.0                           
504                                                0.0                           
505                                                0.0               

In [375]:
#bs_ = BSpline(internal_knots=[25,40,60], intercept=True).fit(age)
ns_ = NaturalSpline(df=7, intercept=False).fit(x)
#bs_.transform(x)
ns_.internal_knots_
#x


array([1.7916    , 2.19757143, 2.7778    , 3.66551429, 4.7211    ,
       6.3338    ])

In [387]:
bs_x = MS([bs('dis', df=4, intercept=True)]).fit(myData)
bs_x.transform(myData)


,intercept,"bs(dis, df=4)[0]","bs(dis, df=4)[1]","bs(dis, df=4)[2]","bs(dis, df=4)[3]"
0,1.0,0.390292,0.431315,0.158883,0.019509
1,1.0,0.275943,0.443723,0.237839,0.042495
2,1.0,0.275943,0.443723,0.237839,0.042495
3,1.0,0.167699,0.409211,0.332846,0.090244
4,1.0,0.167699,0.409211,0.332846,0.090244
...,...,...,...,...,...
501,1.0,0.675286,0.283262,0.039607,0.001846
502,1.0,0.716213,0.252862,0.029758,0.001167
503,1.0,0.742739,0.232219,0.024201,0.000841
504,1.0,0.694296,0.269366,0.034835,0.001502


In [341]:
ns_x = MS([ns('dis', df=3)]).fit(myData)
ns_x.transform(myData)


,intercept,"ns(dis, df=3)[0]","ns(dis, df=3)[1]","ns(dis, df=3)[2]"
0,1.0,0.097671,0.599547,-0.317991
1,1.0,0.281544,0.513711,-0.234343
2,1.0,0.281544,0.513711,-0.234343
3,1.0,0.409347,0.435305,-0.111501
4,1.0,0.409347,0.435305,-0.111501
...,...,...,...,...
501,1.0,-0.201423,0.578114,-0.321047
502,1.0,-0.198985,0.526684,-0.292489
503,1.0,-0.191260,0.487155,-0.270536
504,1.0,-0.201855,0.555817,-0.308667


In [329]:
poly_x = MS([poly('dis', degree=15)]).fit(myData)
M = sm.OLS(y, poly_x.transform(myData)).fit()
summarize(M)
M.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    nox   R-squared:                       0.736
Model:                            OLS   Adj. R-squared:                  0.728
Method:                 Least Squares   F-statistic:                     91.26
Date:                Sat, 07 Oct 2023   Prob (F-statistic):          3.69e-131
Time:                        13:50:51   Log-Likelihood:                 710.40
No. Observations:                 506   AIC:                            -1389.
Df Residuals:                     490   BIC:                            -1321.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
intercept                    0.5547      0.003    206.595      0.000       0.549       0.560
poly(dis, degree=15)[0]     -2.0031      0.060    -33.166      0.000      -2.122      -1.884
poly(dis, degree=15)[1]      0.8563      0.060     14.179      0.000       0.738       0.975
poly(dis, degree=15)[2]     -0.3180      0.060     -5.266      0.000      -0.437      -0.199
poly(dis, degree=15)[3]      0.0335      0.060      0.555      0.579      -0.085       0.152
poly(dis, degree=15)[4]      0.1330      0.060      2.202      0.028       0.014       0.252
poly(dis, degree=15)[5]     -0.1924      0.060     -3.186      0.002      -0.311      -0.074
poly(dis, degree=15)[6]      0.1696      0.060      2.809      0.005       0.051       0.288
poly(dis, degree=15)[7]     -0.1177      0.060     -1.949      0.052      -0.236       0.001
poly(dis, degree=15)[8]      0.0479      0.060      0.794      0.428      -0.071       0.167
poly(dis, degree=15)[9]     -0.0341      0.060     -0.564      0.573      -0.153       0.085
poly(dis, degree=15)[10]     0.0018      0.060      0.031      0.976      -0.117       0.121
poly(dis, degree=15)[11]    -0.0484      0.060     -0.802      0.423      -0.167       0.070
poly(dis, degree=15)[12]     0.1314      0.060      2.175      0.030       0.013       0.250
poly(dis, degree=15)[13]     0.0332      0.060      0.549      0.583      -0.086       0.152
poly(dis, degree=15)[14]    -0.1552      0.060     -2.570      0.010      -0.274      -0.037
==============================================================================
Omnibus:                       59.434   Durbin-Watson:                   0.347
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               79.847
Skew:                           0.862   Prob(JB):                     4.59e-18
Kurtosis:                       3.902   Cond. No.                         22.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [330]:
diff = (M.predict(poly_x.transform(myData)) - y)
RSS = np.sum(np.square(diff))
round(RSS, 2)

1.79

In [331]:
d = {'dis': [6, 6]}
test_data = pd.DataFrame(data=d)
test_result = M.predict(poly_x.transform(test_data))
np.round(test_result, 2)

0    0.45
1    0.45
dtype: float64

## scikit learn

In [206]:
X = np.array([x]).T
Y = np.array([y]).T
#print(X)
#print(Y)

In [207]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


In [263]:
#X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
# y = 1 * x_0 + 2 * x_1 + 3

poly_X_fit = PolynomialFeatures(degree=3).fit(X)
#poly_X = poly_X_fit.transform(X)

reg = LinearRegression().fit(poly_X_fit.transform(X), Y)
#print("reg.score:", reg.score(X, Y))
#print("mean_squared_error:", mean_squared_error(Y, reg.predict(X)))
print("reg.coef_:", reg.coef_)
print("reg.intercept_:", reg.intercept_)
reg.get_params()
print(reg)

reg.coef_: [[ 0.         -0.18208169  0.02192766 -0.000885  ]]
reg.intercept_: [0.93412807]
LinearRegression()


In [264]:
y_pred = reg.predict(poly_X_fit.transform(X))

#y_pred

In [265]:
np.linalg.norm(Y - y_pred)**2
#z = np.linalg.norm(Y - y_pred)
#z*z

1.9341067071790703

In [242]:
from scipy.spatial import distance
# a = (1, 2, 3)
# b = (4, 5, 6)
#distance.euclidean(a, b)
dist = distance.euclidean(y, y_pred.T[0])


1.9341067071790703

In [175]:
diff = (y_pred - Y)
RSS = np.sum(np.square(diff))
round(RSS, 2)

1.93

In [85]:
d = {'dis': [6, 6]}
test_data = pd.DataFrame(data=d)
test_result = M.predict(poly_x.transform(test_data))
np.round(test_result, 2)

0      4.0900
1      4.9671
2      4.9671
3      6.0622
4      6.0622
        ...  
501    2.4786
502    2.2875
503    2.1675
504    2.3889
505    2.5050
Name: dis, Length: 506, dtype: float64

In [197]:
x = np.array([1, 2, 3, 4, 5, 6])
print('beginning x:\n', x)
x_reshape = x.reshape((-1, 1, 2))
print('reshaped x:\n', x_reshape)


beginning x:
 [1 2 3 4 5 6]
reshaped x:
 [[[1 2]]

 [[3 4]]

 [[5 6]]]


In [403]:
np.linspace(0, 1.0, num=5)

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

In [401]:
(np.logspace(2, 3, num=5))

array([ 100.        ,  177.827941  ,  316.22776602,  562.34132519,
       1000.        ])

In [404]:
10**(np.linspace(0, 1.0, num=5))

array([ 1.        ,  1.77827941,  3.16227766,  5.62341325, 10.        ])

In [405]:
(np.logspace(0, 1, num=5))

array([ 1.        ,  1.77827941,  3.16227766,  5.62341325, 10.        ])

In [430]:
from numpy import array
from numpy.linalg import eig
# define matrix
A = array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(A)
# calculate eigendecomposition
values, vectors = eig(A)
print(values)
print(vectors.T)

print("------")
for i in range(3):
    vec = vectors.T[i]
    val = values[i]
    print(vec)
    print(val)
    print(vec * val)
    print(np.matmul(A, vec))
    
    print(np.matmul(A, vec) / vec)
    
    print("*********")
#print(vectors.T[0])
print(vectors.T[2]*-9.75918483e-16)

for i in range(3):
    print(np.sum(np.square(vectors.T[i])))
    #print(np.sum(np.square(vectors[i])))

[[1 2 3]
 [4 5 6]
 [7 8 9]]
[ 1.61168440e+01 -1.11684397e+00 -8.58274334e-16]
[[-0.23197069 -0.52532209 -0.8186735 ]
 [-0.78583024 -0.08675134  0.61232756]
 [ 0.40824829 -0.81649658  0.40824829]]
------
[-0.23197069 -0.52532209 -0.8186735 ]
16.11684396980703
[ -3.73863537  -8.46653421 -13.19443305]
[ -3.73863537  -8.46653421 -13.19443305]
[16.11684397 16.11684397 16.11684397]
*********
[-0.78583024 -0.08675134  0.61232756]
-1.1168439698070434
[ 0.87764976  0.09688771 -0.68387434]
[ 0.87764976  0.09688771 -0.68387434]
[-1.11684397 -1.11684397 -1.11684397]
*********
[ 0.40824829 -0.81649658  0.40824829]
-8.582743335036247e-16
[-3.50389029e-16  7.00778059e-16 -3.50389029e-16]
[-6.66133815e-16 -4.44089210e-16 -4.44089210e-16]
[-1.63168795e-15  5.43895982e-16 -1.08779196e-15]
*********
[-3.98417052e-16  7.96834105e-16 -3.98417052e-16]
0.9999999999999996
1.0
1.0000000000000004


## 

In [434]:
Q = vectors
# create inverse of eigenvectors matrix
R = np.linalg.inv(Q)
# create diagonal matrix from eigenvalues
L = np.diag(values)
# reconstruct the original matrix
B = Q.dot(L).dot(R)
print(B)

[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]
